In [30]:
#author: James Chan © 2018
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [42]:
#We are concerned about tech sentiment, QQQ will be our stock (ETF) of interest.
df = pd.read_csv('QQQ.csv')

In [62]:
'''
Assumptions:
1. We assume we enter buy order at market close, hence using adjusted close. 
2. The news 
'''
df = df[['Date','Open','Close']]
df_open_return = pd.DataFrame((df['Close'] - df['Open']) / df['Open'], columns = ['Open Return'])
df_close_return = pd.DataFrame((df['Open'] - df['Close'].shift(1)) / df['Close'].shift(1), columns = ['Close Return'])    

In [63]:
df.join((df_open_return, df_close_return))

,Date,Open,Close,Open Return,Close Return
0,1999-03-10,51.125000,51.062500,-0.001222,NaN
1,1999-03-11,51.437500,51.312500,-0.002430,0.007344
2,1999-03-12,51.125000,50.062500,-0.020782,-0.003654
3,1999-03-15,50.437500,51.500000,0.021066,0.007491
4,1999-03-16,51.718750,51.937500,0.004230,0.004248
5,1999-03-17,51.937500,51.562500,-0.007220,0.000000
6,1999-03-18,51.500000,52.562500,0.020631,-0.001212
7,1999-03-19,53.250000,51.218750,-0.038146,0.013080
8,1999-03-22,51.437500,50.593750,-0.016403,0.004271
9,1999-03-23,50.312500,48.812500,-0.029814,-0.005559


In [2]:
#author: James Chan © 2018
import requests
#using FAANG not only because it's a cool buzzword, but they are also the tech market drivers
companies = ['facebook','apple','amazon','netflix','google']
companies = ['amazon']
start_date = '2018-07-25'
end_date = '2018-07-28'
news_source = 'wsj.com'
apikey = '23e7e5c41b6443f1bb556ec312275aff'
query_urls = []
for company in companies:
    url = 'https://newsapi.org/v2/everything?q={}&from={}&to={}&domains={}&apiKey={}'\
            .format(company, start_date, end_date, news_source, apikey)
    query_urls.append(url)


#data provided by https://newsapi.org <----these guys are awesome :-)!
for url in query_urls:
    response = requests.get(url)
    news = response.json()

In [4]:
# pandas = 
#data preprocess:
#1. use description only, remove entries that don't actually have the company name mentioned in the description
#2. need to offset time by -4 hours to get ET time.  e.g., 1:00pm ET (summer) is shown as 17:00.
#3. bin the time to market open and market close per day.
for article in news['articles']:
    print(article['title'])
    print(article['description'])
    print()
    print(article['publishedAt'])

Amazon Continues Profit Streak
Amazon continued its streak of profitability, easily topping $1 billion for the third straight quarter, as the company extends its dominance in online retailing and draws gains from its flourishing cloud-computing business.
{'source': {'id': 'the-wall-street-journal', 'name': 'The Wall Street Journal'}, 'author': 'Laura Stevens', 'title': 'Amazon Continues Profit Streak', 'description': 'Amazon continued its streak of profitability, easily topping $1 billion for the third straight quarter, as the company extends its dominance in online retailing and draws gains from its flourishing cloud-computing business.', 'url': 'https://www.wsj.com/articles/amazon-continues-profit-streak-1532635804', 'urlToImage': 'https://images.wsj.net/im-19760/social', 'publishedAt': '2018-07-26T20:10:06Z'}


2018-07-26T20:10:06Z
How Sellers Trick Amazon To Boost Sales
The digital giant battles click farms, reviewers-for-hire and other scams as merchants try to outsmart its produc